##### Experiment 1

Model Size: 253MB

Original Repo： Hugging Face.

Framework: PyTorch.

NLP Task: Text Classification

Host Config:

```
runtime: python
api_version: '1'
env: flexible
threadsafe: true
manual_scaling:
  instances: 1
resources:
  cpu: 1
  memory_gb: 1
  disk_size_gb: 20
liveness_check:
  initial_delay_sec: 300
  check_interval_sec: 300
  timeout_sec: 4
  failure_threshold: 4
  success_threshold: 2
readiness_check:
  check_interval_sec: 500
  timeout_sec: 4
  failure_threshold: 2
  success_threshold: 2
  app_start_timeout_sec: 300
```

In [1]:
gae_distbert = 'https://distilbert-dot-gft-bike-insurance.nw.r.appspot.com/run-zerohot'
data_distbert = {
    "sentence": "I am not unhappy about the service you provided.",
    "labels": "happy,unhappy,neutral"
  }
import subprocess
import json
import requests

In [3]:
output = subprocess.check_output("gcloud auth print-identity-token", shell=True, text=True)
access_token = "Bearer " + output[:-1]
headers = {'Authorization': access_token}
response = requests.post(gae_distbert, 
                         headers=headers, 
                         json=data_distbert)
print(response.text)

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37



In [10]:
import datetime
import time

tot_pred_time = datetime.timedelta(0)
execution_time = []

for i in range(20):
    print('Epoch: ', i)
    time_start = datetime.datetime.now()
    output = subprocess.check_output("gcloud auth print-identity-token", shell=True, text=True)
    access_token = "Bearer " + output[:-1]
    headers = {'Authorization': access_token}
    response = requests.post(gae_distbert, 
                                 headers=headers, 
                                 json=data_distbert)
    print(response.text)
    time_end = datetime.datetime.now()
    time_diff = time_end - time_start
    print("the time for making the API call is ", time_diff)
    tot_pred_time += time_diff
    execution_time.append(time_diff)
    time.sleep(20)
    
print("the average time for making an API call is ", tot_pred_time/20)

Epoch:  0
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.798133
Epoch:  1
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.755602
Epoch:  2
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.194144
Epoch:  3
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.659013
Epoch:  4
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.616278
Epoch:  5
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.016494
Epoch:  6
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.117504

In [17]:
import numpy as np
print(np.min(execution_time))
print(np.mean(execution_time))
print(np.max(execution_time))

0:00:03.807304
0:00:04.364096
0:00:05.373103


### Experiment 2

Host Config:

```
runtime: python
api_version: '1'
env: flexible
threadsafe: true
manual_scaling:
  instances: 3
resources:
  cpu: 1
  memory_gb: 1
  disk_size_gb: 20
```

In [20]:
import time
import os                                                                       
import threading

def num_of_requests_processed_within_a_period(secs):
    num = 0
    stop_time = time.time() + secs
    while time.time() < stop_time:
        output = subprocess.check_output("gcloud auth print-identity-token", shell=True, text=True)
        access_token = "Bearer " + output[:-1]
        headers = {'Authorization': access_token}
        response = requests.post(gae_distbert, 
                                     headers=headers, 
                                     json=data_distbert)
        print(response.text)
        num += 1
    return num

In [22]:
import concurrent.futures
ls_num_threads = list(range(1, 5))
tot_num_requests_processed = []
num_secs = 20
print('The throughput is tested within a duration of {}s'.format(str(num_secs)))

for num_threads in ls_num_threads:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(num_of_requests_processed_within_a_period, num_secs) for param in [num_secs]*num_threads]
        results = [f.result() for f in futures]
        tot_req = sum(results)
        tot_num_requests_processed.append(tot_req)
        print('num_threads is: {};'.format(str(num_threads)), 
              'tot number of requests processed in {}s are {}'.format(str(num_secs), 
                                                                      str(tot_req)))

The throughput is tested within a duration of 20s
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

num_threads is: 1; tot number of requests processed in 20s are 4
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -

### Experiment 3

Host Config:

```
runtime: python38
service: distilbert

instance_class: F4_1G

automatic_scaling:
  target_cpu_utilization: 0.65
  max_instances: 1

entrypoint: gunicorn -b :$PORT -w 8 main:app --timeout 120
```

In [23]:
gae_distbert_standard = 'https://distilbert-dot-gft-bike-insurance.nw.r.appspot.com/run-zerohot'
data_distbert_standard = {
    "sentence": "I am not unhappy about the service you provided.",
    "labels": "happy,unhappy,neutral"
  }

In [25]:
output = subprocess.check_output("gcloud auth print-identity-token", shell=True, text=True)
access_token = "Bearer " + output[:-1]
headers = {'Authorization': access_token}
response = requests.post(gae_distbert_standard, 
                         headers=headers, 
                         json=data_distbert_standard)
print(response.text)

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37



In [26]:
import datetime
import time

tot_pred_time = datetime.timedelta(0)
execution_time = []

for i in range(20):
    print('Epoch: ', i)
    time_start = datetime.datetime.now()
    output = subprocess.check_output("gcloud auth print-identity-token", shell=True, text=True)
    access_token = "Bearer " + output[:-1]
    headers = {'Authorization': access_token}
    response = requests.post(gae_distbert_standard, 
                                 headers=headers, 
                                 json=data_distbert_standard)
    print(response.text)
    time_end = datetime.datetime.now()
    time_diff = time_end - time_start
    print("the time for making the API call is ", time_diff)
    tot_pred_time += time_diff
    execution_time.append(time_diff)
    time.sleep(20)
    
print("the average time for making an API call is ", tot_pred_time/20)

Epoch:  0
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:19.688789
Epoch:  1
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:04.119564
Epoch:  2
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:03.700067
Epoch:  3

<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>500 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered an error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>

the time for making the API call is  0:00:06.725635
Epoch:  4
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

the time for making the API call is  0:00:19.384260
Epoch:  5
happy 	 similarity: 0.7
neutral 

In [27]:
import numpy as np
print(np.min(execution_time))
print(np.mean(execution_time))
print(np.max(execution_time))

0:00:03.181933
0:00:08.910389
0:00:19.688789


In [28]:
import time
import os                                                                       
import threading

def num_of_requests_processed_within_a_period(secs):
    num = 0
    stop_time = time.time() + secs
    while time.time() < stop_time:
        output = subprocess.check_output("gcloud auth print-identity-token", shell=True, text=True)
        access_token = "Bearer " + output[:-1]
        headers = {'Authorization': access_token}
        response = requests.post(gae_distbert_standard, 
                                     headers=headers, 
                                     json=data_distbert_standard)
        print(response.text)
        num += 1
    return num

In [29]:
import concurrent.futures
ls_num_threads = list(range(1, 5))
tot_num_requests_processed = []
num_secs = 20
print('The throughput is tested within a duration of {}s'.format(str(num_secs)))

for num_threads in ls_num_threads:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(num_of_requests_processed_within_a_period, num_secs) for param in [num_secs]*num_threads]
        results = [f.result() for f in futures]
        tot_req = sum(results)
        tot_num_requests_processed.append(tot_req)
        print('num_threads is: {};'.format(str(num_threads)), 
              'tot number of requests processed in {}s are {}'.format(str(num_secs), 
                                                                      str(tot_req)))

The throughput is tested within a duration of 20s
happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37


<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>500 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered an error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>

happy 	 similarity: 0.7
neutral 	 similarity: -0.26
unhappy 	 similarity: -0.37

num_threads is: 1; tot number of requests processed in 20s are 4

<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>500 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered an error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></h

Saw Memory issue during test

```
{
textPayload: "Exceeded hard memory limit of 2048 MB with 2067 MB after servicing 1 requests total. Consider setting a larger instance class in app.yaml."
insertId: "608aec3a0001b8cf38da9263"
resource: {2}
timestamp: "2021-04-29T17:26:18.112847Z"
labels: {1}
logName: "projects/gft-bike-insurance/logs/syslog"
receiveTimestamp: "2021-04-29T17:26:18.117269104Z"
}
```